In [1]:
import pandas as pd
import re
import numpy as np
from scipy.stats import skew
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("score1694.csv")
del df["Unnamed: 0"]

In [2]:
df.columns.values

array(['글쎄요', '보고싶어요', '참여인원', '평점', 'title', 'y', '런타임', '장르', '개봉일',
       '등급', '개봉년도', '개봉월'], dtype=object)

In [3]:
df.head()

,글쎄요,보고싶어요,참여인원,평점,title,y,런타임,장르,개봉일,등급,개봉년도,개봉월
0,11,54,30,9.60,10분 (10 Minutes),0,93,['드라마'],['2014.04.24'],12세,2014,4
1,0,0,277,8.81,10억 (A Million),1,114,"['모험', '스릴러']",['2009.08.06'],15세,2009,8
2,4,10,19,8.79,125 전승철 (125 JEON Seung-Chul),0,21,['드라마'],[],NaN,0,0
3,0,0,63,7.60,1724 기방난동사건,0,103,"['코미디', '액션', '모험']",['2008.12.03'],15세,2008,12
4,18,153,32,9.09,1789 바스티유의 연인들 (1789 Les Amants De La Bastille),0,129,"['공연실황', '뮤지컬']",['2014.09.18'],전체,2014,9


## object 변수 따로 빼기(df_cat)

In [4]:
cats =[]
for col in df.columns.values:
    if df[col].dtype == 'object':
        cats.append(col)
        
df_cont = df.drop(cats, axis=1)
df_cont = df.drop("y", axis=1)
y = df["y"]
df_cat = df[cats]

## object변수 더미화하기

In [5]:
for col in df_cat.columns.values:
    print(col)
    df_cat[col] = LabelEncoder().fit_transform(df_cat[col])
    num_cols = df_cat[col].max()
    
    for i in range(num_cols):
        col_name = col + '_' + str(i)
        df_cat[col_name] = df_cat[col].apply(lambda x: 1 if x == i else 0)
    
    df_cat = df_cat.drop(col, axis=1)

title


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


장르
개봉일
등급


TypeError: '<' not supported between instances of 'str' and 'float'

## 아웃라이어 처리

In [ ]:
def is_outlier(points, thresh=3.5):
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

## 수치형 변수 처리

In [6]:
for col in df_cont.columns.values:
    if sum(df_cont[col]==0) > 0:
        median = df_cont[col].median()
        idx = np.where(df_cont[col]==0)[0]
        df_cont[col].iloc[idx] = median
        
        outliers = np.where(is_outlier(df_cont[col]))
        df_cont[col].iloc[outliers] = median
        
        if skew(df_cont[col]) > 0.75:
            df_cont[col] = np.log(df_cont[col])
            df_cont[col] = df_cont[col].apply(lambda x: 0 if x == -np.inf else x)
        
        df_cont[col] = Normalizer().fit_transform(df_cont[col].reshape(1,-1))[0]

C:\Users\user\Anaconda3\lib\site-packages\pandas\core\indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [8]:
df_new = df_cont.join(df_cat)

In [9]:
df_new = df_new.drop(["등급","개봉년도","개봉월"],axis=1)

## 데이터 나누기

In [10]:
from sklearn.cross_validation import train_test_split

train_X, test_X = train_test_split(df_new, test_size = 0.3, random_state=1) # 독립변수만
train_Y, test_Y = train_test_split(y, test_size = 0.3, random_state=1) # 종속변수만

C:\Users\user\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

sc = StandardScaler()
sc.fit(train_X)
train_X_std = sc.transform(train_X)
test_X_std = sc.transform(test_X)

## 간단한 모델들

In [366]:
from sklearn.linear_model import LogisticRegression

ml = LogisticRegression()
ml.fit(train_X_std, train_Y)
pred = ml.predict(test_X_std)
print(accuracy_score(test_Y, pred)) 

ml2 = LogisticRegression()
ml2.fit(train_X, train_Y)
pred2 = ml2.predict(test_X)
print(accuracy_score(test_Y, pred2)) 

0.588235294118
0.558823529412


In [16]:
from sklearn import tree
tree = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=0)
tree.fit(train_X_std, train_Y)

pred_tr = tree.predict(test_X_std)
#print('Accuracy: %.2f' % accuracy_score(test_Y, pred_tr))
scores = cross_val_score(estimator=tree,X=train_X, y=train_Y, cv=10,n_jobs=1)
print('CV accuracy scores: %s' % scores)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy scores: [ 0.625       0.5         0.875       0.625       0.5         0.5         0.5
  0.57142857  0.57142857  0.71428571]
CV accuracy: 0.598 +/- 0.115


In [17]:
from sklearn.svm import SVC
svc1 = SVC(kernel="rbf",gamma=0.01, random_state=0)
svc1.fit(train_X_std, train_Y)
pred_svc1 = svc1.predict(test_X_std)
#print('Accuracy: %.2f' % accuracy_score(test_Y, pred_svc1))
scores = cross_val_score(estimator=svc1,X=train_X, y=train_Y, cv=10,n_jobs=1)
print('CV accuracy scores: %s' % scores)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy scores: [ 0.5         0.5         0.5         0.5         0.5         0.5         0.5
  0.57142857  0.57142857  0.57142857]
CV accuracy: 0.521 +/- 0.033


In [18]:
from sklearn.neighbors import KNeighborsClassifier
knn1 = KNeighborsClassifier(n_neighbors=7, p=2, metric='minkowski' )
knn1.fit(train_X, train_Y)
pred_knn1 = knn1.predict(test_X)
print('Accuracy: %.2f' % accuracy_score(test_Y, pred_knn1))
scores = cross_val_score(estimator=knn1,X=train_X, y=train_Y, cv=10,n_jobs=1)
print('CV accuracy scores: %s' % scores)
print('CV accuracy: %.3f +/- %.3f' `% (np.mean(scores), np.std(scores)))

Accuracy: 0.76
CV accuracy scores: [ 0.625       0.5         0.75        0.5         0.75        0.625       0.75
  0.71428571  0.85714286  0.71428571]
CV accuracy: 0.679 +/- 0.109


In [20]:
from sklearn.ensemble import RandomForestClassifier
rf1 = RandomForestClassifier(criterion="entropy", n_estimators = 100, n_jobs=2, random_state=0)
rf1.fit(train_X_std, train_Y)
pred_rf1 = rf1.predict(test_X_std) 
#print('Accuracy: %.2f' % accuracy_score(test_Y, pred_rf1))
scores = cross_val_score(estimator=rf1,X=train_X, y=train_Y, cv=10,n_jobs=1)
print('CV accuracy scores: %s' % scores)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy scores: [ 0.625       0.375       0.625       0.625       0.75        0.75        0.625
  0.57142857  0.85714286  0.71428571]
CV accuracy: 0.652 +/- 0.123


In [14]:
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.cross_validation import cross_val_score
scorer = make_scorer(mean_squared_error, False)

RF = RandomForestClassifier(n_estimators=500, n_jobs=2, random_state=0)

In [15]:
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(estimator=RF, 
                         X=train_X, 
                         y=train_Y, 
                         cv=10,
                         n_jobs=1)
print('CV accuracy scores: %s' % scores)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

CV accuracy scores: [ 0.625       0.375       0.75        0.5         0.75        0.75        0.625
  0.57142857  0.85714286  0.71428571]
CV accuracy: 0.652 +/- 0.135


In [253]:
RF.fit(train_X, train_Y)
pred_RF = RF.predict(test_X)
print('Accuracy: %.2f' % accuracy_score(test_Y, pred_RF))
coef = pd.Series(RF.feature_importances_, index = train_X.columns).sort_values(ascending=False)
coef

Accuracy: 0.74


보고싶어요        0.123435
기대지수         0.119875
관심인원         0.089849
글쎄요          0.083259
런타임          0.077495
참여인원         0.076977
id_length    0.070460
평점           0.065753
배우수          0.062916
조연수          0.055116
주연수          0.026210
개봉년도_0       0.017727
개봉월_3        0.012967
등급_0         0.012371
등급_1         0.012049
개봉년도_2       0.010577
개봉년도_1       0.009940
개봉월_2        0.009717
개봉월_4        0.009205
개봉월_5        0.009167
개봉월_8        0.008696
개봉월_1        0.007935
개봉월_7        0.007173
개봉월_10       0.005301
개봉월_0        0.005232
개봉월_6        0.004288
개봉월_9        0.003679
등급_2         0.002630
미스터리         0.000000
판타지          0.000000
코미디          0.000000
전쟁           0.000000
액션           0.000000
스릴러          0.000000
서스펜스         0.000000
범죄           0.000000
SF           0.000000
가족           0.000000
공포           0.000000
느와르          0.000000
다큐멘터리        0.000000
드라마          0.000000
로맨스          0.000000
멜로           0.000000
모험           0.000000
서사        

In [261]:
X = train_X
X["y"] = y

# 종속변수과 상관관계 높은 변수 보기
corr = X.corr()
corr.sort_values(["y"], ascending=False, inplace=True)
print(corr.y)

y            1.000000e+00
기대지수         3.323805e-01
개봉년도_0       2.688474e-01
런타임          2.387938e-01
배우수          2.320634e-01
보고싶어요        2.259708e-01
등급_0         2.057911e-01
조연수          2.022429e-01
관심인원         1.696675e-01
개봉월_8        1.479591e-01
평점           1.470488e-01
개봉월_5        1.262571e-01
개봉년도_2       1.106368e-01
개봉월_6        1.083086e-01
주연수          1.054438e-01
참여인원         8.312255e-02
개봉월_9        7.501724e-02
개봉월_10       6.301961e-02
개봉월_7        5.463825e-02
개봉월_1        1.133462e-02
드라마         -5.627480e-17
개봉월_4       -3.287980e-02
등급_2        -5.931596e-02
개봉월_0       -8.563933e-02
등급_1        -8.701726e-02
개봉년도_1      -1.284169e-01
글쎄요         -1.436867e-01
개봉월_2       -1.880817e-01
개봉월_3       -2.169130e-01
id_length   -3.242383e-01
SF                    NaN
가족                    NaN
공포                    NaN
느와르                   NaN
다큐멘터리                 NaN
로맨스                   NaN
멜로                    NaN
모험                    NaN
미스터리        

C:\Users\user\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [21]:
# 상위 몇개 변수에 3승 까지 적용
df_new["기대지수-s2"] = df_new["기대지수"] **2
df_new["기대지수-s3"] = df_new["기대지수"] **3
df_new["기대지수-sq"] = np.sqrt(df_new["기대지수"])

df_new["런타임-s2"] = df_new["런타임"] **2
df_new["런타임-s3"] = df_new["런타임"] **3
df_new["런타임-sq"] = np.sqrt(df_new["런타임"])

df_new["id-s2"] = df_new["id_length"] **2
df_new["id-s3"] = df_new["id_length"] **3
df_new["id-sq"] = np.sqrt(df_new["id_length"])

df_new["보고싶어요-s2"] = df_new["보고싶어요"] **2
df_new["보고싶어요-s3"] = df_new["보고싶어요"] **3
df_new["보고싶어요-sq"] = np.sqrt(df_new["보고싶어요"])

df_new["관심-s2"] = df_new["관심인원"] **2
df_new["관심-s3"] = df_new["관심인원"] **3
df_new["관심-sq"] = np.sqrt(df_new["관심인원"])

In [22]:
from sklearn.cross_validation import train_test_split

train_X, test_X = train_test_split(df_new, test_size = 0.3, random_state=1) # 독립변수만
train_Y, test_Y = train_test_split(y, test_size = 0.3, random_state=1) # 종속변수만

In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

sc = StandardScaler()
sc.fit(train_X)
train_X_std = sc.transform(train_X)
test_X_std = sc.transform(test_X)

In [24]:
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV

ridge = RidgeCV(alphas = [0.01, 0.03, 0.06, 0.1, 0.3, 0.6, 1, 3 , 6, 10, 30,60])
ridge.fit(train_X, train_Y)
alpha = ridge.alpha_
print("Best alpha : ", alpha)

ridge = RidgeCV(alphas = [alpha *.6, alpha * .65, alpha * .7,alpha * .75,
                         alpha * .8, alpha * .85, alpha * .9, alpha * .95,
                         alpha * 1.05, alpha * 1.1, alpha * 1.15, alpha * 1.25,
                         alpha * 1.3, alpha * 1.35, alpha * 1.4], cv=10)
ridge.fit(train_X, train_Y)
alpha = ridge.alpha_
print("Best alpha : ", alpha)
pred_ridge = ridge.predict(test_X)
#print('Accuracy: %.2f' % accuracy_score(test_Y, pred_lasso))

coefs = pd.Series(ridge.coef_, index = train_X.columns)
print("ridge는 변수를 몇개 선택했니? : ", sum(coefs!=0)) 
print("ridge는 변수를 몇개 버렸니? : ", sum(coefs==0))


Best alpha :  60.0
Best alpha :  84.0
ridge는 변수를 몇개 선택했니? :  44
ridge는 변수를 몇개 버렸니? :  17


In [25]:
lasso = LassoCV(alphas = [0.0001, 0.0003, 0.0006, 0.001, 0.003, 0.006, 
                          0.01, 0.03, 0.06, 0.1, 0.3, 0.6, 1])
lasso.fit(train_X, train_Y)
alpha = lasso.alpha_
print("Best alpha : ", alpha)

lasso = LassoCV(alphas = [alpha *.6, alpha * .65, alpha * .7,alpha * .75,
                         alpha * .8, alpha * .85, alpha * .9, alpha * .95,
                         alpha * 1.05, alpha * 1.1, alpha * 1.15, alpha * 1.25,
                         alpha * 1.3, alpha * 1.35, alpha * 1.4],
               max_iter = 50000, cv=10)
lasso.fit(train_X, train_Y)
alpha = lasso.alpha_
print("Best alpha : ", alpha)
pred_lasso = lasso.predict(test_X)
#print('Accuracy: %.2f' % accuracy_score(test_Y, pred_lasso))

coefs = pd.Series(lasso.coef_, index = train_X.columns)
print("lasso는 변수를 몇개 선택했니? : ", sum(coefs!=0)) 
print("lasso는 변수를 몇개 버렸니? : ",sum(coefs==0)) 


C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Best alpha :  1.0


C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Best alpha :  1.4
lasso는 변수를 몇개 선택했니? :  10
lasso는 변수를 몇개 버렸니? :  51


C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [352]:
pred_lasso1 = np.where(pred_lasso > 0.5,1,0)
print('Accuracy: %.2f' % accuracy_score(test_Y, pred_lasso1))

Accuracy: 0.76


In [354]:
pred_ridge1 = np.where(pred_ridge > 0.5, 1, 0)
print('Accuracy: %.2f' % accuracy_score(test_Y, pred_ridge1))

Accuracy: 0.74


In [341]:
#from sklearn.cross_validation import ShuffleSplit
#from sklearn.utils import shuffle
#import xgboost as xgb

In [26]:
scores = cross_val_score(estimator=lasso,X=train_X, y=train_Y, cv=10,n_jobs=1)
print('CV accuracy scores: %s' % scores)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase t

CV accuracy scores: [  3.55106022e-01   2.94013632e-01   2.13601675e-01  -6.40026469e-01
  -2.70614646e+00   1.76332324e-01   7.28378575e-01  -4.44886375e-02
   2.94735517e-01  -3.61667194e+02]
CV accuracy: -36.300 +/- 108.460


C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [ ]:
scores = cross_val_score(estimator=ridge,X=train_X, y=train_Y, cv=10,n_jobs=1)
print('CV accuracy scores: %s' % scores)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))